# Memory reduction

In [ ]:
# Use to reduce the size of dataframes when needed

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2
    NAlist = [] # Keeps track of columns that have missing values filled in.
    for col in props.columns:
        if (props[col].dtype != object) & (props[col].dtype != 'category'):  # Exclude strings

            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)

            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)
            else:
                props[col] = props[col].astype(np.float32)

    mem_usg = props.memory_usage().sum() / 1024**2
    return props


# Datasets

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Change the path if needed
project_path = ''

df_cel = pd.read_csv(project_path + 'data/data_cel_sim.csv')
df_hts = pd.read_csv( project_path + 'data/data_hts_sim.csv')
df_true = pd.read_csv(project_path + 'data/data_true_sim.csv')

print(df_hts.info())
print(df_cel.info())
print(df_true.info())

del df_hts, df_cel, df_true

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      2000 non-null   int64  
 1   Age         2000 non-null   int64  
 2   Homeincome  2000 non-null   int64  
 3   NumHH       2000 non-null   int64  
 4   Distance    2000 non-null   int64  
 5   Home_X      2000 non-null   float64
 6   Home_Y      2000 non-null   float64
 7   Work_X      2000 non-null   float64
 8   Work_Y      2000 non-null   float64
 9   Home_Hcode  2000 non-null   int64  
 10  Work_Hcode  2000 non-null   int64  
 11  Prob        2000 non-null   float64
dtypes: float64(5), int64(7)
memory usage: 187.6 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5895 entries, 0 to 5894
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Distance    5895 non-null   int64  
 1   Home_X      5895 non-null   

# Clustering

In [ ]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering
from scipy.special import rel_entr, kl_div
from scipy.stats import entropy
import warnings
warnings.filterwarnings('ignore')

df_htsz = pd.read_csv(project_path + 'data/data_hts_sim.csv')
df_celz = pd.read_csv(project_path + 'data/data_cel_sim.csv')

# Attributes
att_X = ['Homeincome', 'NumHH', 'Gender', 'Age']
att_Z = ['Home_X', 'Home_Y' , 'Work_X', 'Work_Y']
att_Y = ['Distance']
att_C = ['Cluster']

# JS divergence between two distributions p and q
def js_div(p, q):
    h = 0.5*np.add(p,q)
    return 0.5*rel_entr(p, h) + 0.5*rel_entr(q, h)

# BRICH clustering, any clustering methods can be adopted
def clustering(df, birch_threshold, bf, K, w):
    w_ = [w, w, 1-w, 1-w]
    X = df * w_
    data = preprocessing.MinMaxScaler().fit_transform(X)
    brc = Birch(threshold=birch_threshold, branching_factor=bf, n_clusters=K)
    brc.fit(data)
    labels = brc.labels_
    pred = brc.predict(data)
    df['Cluster'] = pred
    clusters = np.unique(pred)
    return df.drop_duplicates()

# Auxilary attributes (i.e., distance) based on home and work locations
def add_distance(df, N):
    lon1 = np.radians(df['Home_X'])
    lon2 = np.radians(df['Work_X'])
    lat1 = np.radians(df['Home_Y'])
    lat2 = np.radians(df['Work_Y'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    df['Distance'] = np.ceil(6371*2 * np.arcsin(np.sqrt(a)))
    if (N > 1):
        bin_dis = [i*int(12/N)  for i in range(N+1)] + [25,99999]
        label_dis = [i for i in range(1,len(bin_dis)-1)] + [99]
    else:
        bin_dis = [0,99999]
        label_dis = [1]
    df['Distance'] = pd.cut(df['Distance'], bins=bin_dis, labels=label_dis, include_lowest=True)

# Calculate entropy of a distribution
def calculate_entropy(df_celzz):
    df_celzzk_ = df_celzz[att_Y + att_C + att_Z + ['Prob_cel']]
    df_celzzk_['Prob_cel_tt'] = df_celzzk_.groupby(att_Y + att_C)['Prob_cel'].transform('sum')
    df_celzzk_['Count'] = 2.0
    df_celzzk_['Prob_cel'] = df_celzzk_['Prob_cel'] / df_celzzk_['Prob_cel_tt']
    df_celzzk_['entropy'] = df_celzzk_['Prob_cel_tt'] * (-df_celzzk_['Prob_cel'] * np.emath.logn(df_celzzk_['Count'], df_celzzk_['Prob_cel']))
    return df_celzzk_[['entropy']].sum()[0]


# parato check
def find_prareto(sols, sol):
    offset = 4
    sols_ = []
    if len(sols) == 0:
        sols_ = [sol]
    for en in sols:
        if (sol[offset] < en[offset]) & (sol[offset+1] < en[offset+1]):
            if sol not in sols_:
                sols_.append(sol)
        elif (sol[offset] <= en[offset]) & (sol[offset+1] < en[offset+1]):
            if sol not in sols_:
                sols_.append(sol)
        elif (sol[offset] < en[offset]) & (sol[offset+1] <= en[offset+1]):
            if sol not in sols_:
                sols_.append(sol)
        else:
            if (sol[offset] < en[offset]) | (sol[offset+1] < en[offset+1]):
                if sol not in sols_:
                    sols_.append(sol)
            if en not in sols_:
                sols_.append(en)
    return sols_

# basic grid serach for fing the efficient clustering
def find_clustering(df_htsz, df_celz):
    birch_threshold = 0.005
    branching_factor = 10
    div_ = 99999.0
    div1_ = 99999.0
    div3_ = 99999.0
    k_cluster_ = 10
    n_partition_ = 5
    w_cluster_ = 0.5
    z_true_cluster_ = 0
    obj_val_ = 1.0
    df_ = pd.DataFrame()

    solutions = []

    # Number of clusters
    for k_cluster in [i*2 for i in reversed(range(1,20))]:
        # Weights for clustering
        for w_cluster in [i*0.1 for i in range(0,11)]:
            df_htszz = df_htsz.drop(columns=['Distance'])
            df_celzz = df_celz.drop(columns=['Distance'])
            df_hts_ = df_htszz[att_Z].drop_duplicates()
            df_cel_ = df_celzz[att_Z].drop_duplicates()
            df_clustering = pd.concat([df_cel_, df_hts_]).drop_duplicates()
            clusters = clustering(df_clustering, birch_threshold, branching_factor, k_cluster, w_cluster)

            df_htszz = pd.merge(df_htszz, clusters, on=att_Z)
            df_celzz = pd.merge(df_celzz, clusters, on=att_Z)

            # Number of bins/categories (i.e., +2 to convert to the actual number of distance bins)
            for n_partition in range(1,13):
                add_distance(df_htszz, n_partition)
                add_distance(df_celzz, n_partition)

                df_htszzz = df_htszz[att_X + att_Y + att_Z  + att_C + ['Prob']]
                df_celzzz = df_celzz[att_Y + att_Z  + att_C + ['Prob']]

                df_hts = df_htszzz[att_Y + att_C + ['Prob']]
                df_cel = df_celzzz[att_Y + att_C + ['Prob']]

                df_hts['Prob_hts'] = df_hts.groupby(att_Y + att_C)['Prob'].transform('sum')
                df_hts = df_hts.drop(columns=['Prob'])
                df_hts = df_hts.drop_duplicates()
                df_cel['Prob_cel'] = df_cel.groupby(att_Y + att_C)['Prob'].transform('sum')
                df_cel = df_cel.drop(columns=['Prob'])
                df_cel = df_cel.drop_duplicates()

                df = pd.merge(df_cel, df_hts, on=att_Y + att_C)
                df2 = pd.merge(df_cel, df_hts, how='outer', on=att_Y + att_C).reset_index(drop=True)
                df2 = df2.replace(np.nan, 0)
                z_true_cluster = len(df)
                p_cluster = df['Prob_cel'].sum()
                div1 = np.sum(js_div(df2['Prob_hts'], df2['Prob_cel']))

                df_celzz_ = df_celzzz[att_Z + att_Y + att_C + ['Prob']]
                df_celzz_['Prob_cel'] = df_celzz_.groupby(att_Z + att_Y + att_C)['Prob'].transform('sum')
                df_celzz_ = df_celzz_.drop(columns=['Prob'])
                df_celzz_ = df_celzz_.drop_duplicates()
                div2 = calculate_entropy(df_celzz_)

                epsilon = 0.98
                if (p_cluster > epsilon):
                    sol = [k_cluster,
                           n_partition,
                           w_cluster,
                           z_true_cluster,
                           div1,
                           div2,
                           p_cluster]
                    solutions_ = find_prareto(solutions, sol)
                    solutions = solutions_

    return solutions

# Get the parameters for clustering
solutions = find_clustering(df_htsz, df_celz)
df = pd.DataFrame(solutions, columns=list('abcdefg'))
df[['ee']] = (df[['e']]-df['e'].min())/[max(0.00001,df['e'].max() - df['e'].min())]
df[['ff']] = (df[['f']]-df['f'].min())/[max(0.00001,df['f'].max() - df['f'].min())]
df['r'] = df['ee'] + df['ff']

df = df.sort_values(by=['r']).iloc[0]
result = df[['a', 'b', 'c', 'd']].to_list()
k_cluster = result[0]
w_cluster = result[2]
n_partition = result[1]

# Show the parameters
print('Optimized hyperparameters:')
print('k_cluster: ' + str(k_cluster))
print('w_cluster: ' + str(w_cluster))
print('n_partition: ' + str(n_partition))


# Cluster-based data fusion

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pyomo.environ as pyo
from pyomo.environ import *
from scipy.special import rel_entr, kl_div
import math
import warnings
warnings.filterwarnings('ignore')

# Attributes
att_X = ['Gender', 'Age', 'Homeincome', 'NumHH']
att_Y = ['Distance']
att_Z = ['Home_X', 'Home_Y' , 'Work_X', 'Work_Y']
att_C = ['Cluster']
att_HC = ['Home_Hcode', 'Work_Hcode']

# JS divergence
def js_div(p, q):
    h = 0.5*np.add(p,q)
    return 0.5*rel_entr(p, h) + 0.5*rel_entr(q, h)

# Clustering
def clustering(df, birch_threshold, bf, K, w):
    w_ = [w, w, 1-w, 1-w]
    X = df * w_
    data = preprocessing.MinMaxScaler().fit_transform(X)
    brc = Birch(threshold=birch_threshold, branching_factor=bf, n_clusters=K)
    brc.fit(data)
    labels = brc.labels_
    pred = brc.predict(data)
    df['Cluster'] = pred
    clusters = np.unique(pred)
    return df.drop_duplicates()


# Update auxilary attributes
def add_distance(df, N):
    lon1 = np.radians(df['Home_X'])
    lon2 = np.radians(df['Work_X'])
    lat1 = np.radians(df['Home_Y'])
    lat2 = np.radians(df['Work_Y'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    df['Distance'] = np.ceil(6371*2 * np.arcsin(np.sqrt(a)))
    if (N > 1):
        bin_dis = [i*int(12/N)  for i in range(N+1)] + [25,99999]
        label_dis = [i for i in range(1,len(bin_dis)-1)] + [99]
    else:
        bin_dis = [0,99999]
        label_dis = [1]
    df['Distance'] = pd.cut(df['Distance'], bins=bin_dis, labels=label_dis, include_lowest=True)

# Subproblem P_{op_fusion}
def prob_XYC_optim(df_hts_XYC, df_cel_YC):
    df_hts_XYC_opt = pd.DataFrame()
    cluster_loss = 0
    print('Subproblem ', len(df_cel_YC))
    for index, row in df_cel_YC.iterrows():
        df_hts_XYC_ = df_hts_XYC[(df_hts_XYC['Distance'] == row['Distance']) &
                                (df_hts_XYC['Cluster'] == row['Cluster'])].reset_index(drop=True)
        prob_YC = row['Prob_YC_cel']
        N = len(df_hts_XYC_)
        obj_val = 0
        if N > 0:
            model = pyo.ConcreteModel()
            model.prob = pyo.Var(range(N), initialize = prob_YC/N, bounds = (0, prob_YC))
            model.OBJ = pyo.Objective(expr = np.sum(js_div(df_hts_XYC_['Prob_XYC_hts'],
                                        pd.Series(model.prob.extract_values()))), sense = pyo.minimize)
            model.const = pyo.Constraint(expr = sum(model.prob[n] for n in range(N)) == prob_YC)
            opt_ = pyo.SolverFactory('ipopt')
            opt_.solve(model)
            obj_val += pyo.value(model.OBJ)
            df_hts_XYC_['Prob_XYC_opt'] = pd.Series(model.prob.extract_values())
            df_hts_XYC_ = df_hts_XYC_.drop(columns=['Prob_XYC_hts'])
            df_hts_XYC_opt = pd.concat([df_hts_XYC_opt, df_hts_XYC_])
        else: cluster_loss += prob_YC

    df_hts_XYC_opt = df_hts_XYC_opt[df_hts_XYC_opt['Prob_XYC_opt'] > 0]
    df_hts_XYC_opt[['Prob_XYC_opt']] = df_hts_XYC_opt[['Prob_XYC_opt']] / df_hts_XYC_opt[['Prob_XYC_opt']].sum()
    return cluster_loss, df_hts_XYC_opt

# Calculate the fused distribution
def prob_XYZ(df_hts_, df_cel_, clusters):
    df_hts = pd.merge(df_hts_, clusters, on=att_Z)
    df_cel = pd.merge(df_cel_, clusters, on=att_Z)

    df_cel_YZC = df_cel[att_Y + att_HC + att_Z + att_C + ['Prob']]
    df_cel_YZC.rename(columns = {'Prob':'Prob_YZC_cel'}, inplace = True)

    df_cel_YC = df_cel_YZC[att_Y + att_C + ['Prob_YZC_cel']]
    df_cel_YC['Prob_YC_cel'] = df_cel_YC.groupby(att_Y + att_C)['Prob_YZC_cel'].transform('sum')
    df_cel_YC = df_cel_YC.drop(columns=['Prob_YZC_cel'])
    df_cel_YC = df_cel_YC.drop_duplicates()

    df_hts_XYC = df_hts[att_X + att_Y + att_C + ['Prob']]
    df_hts_XYC['Prob'] = df_hts_XYC.groupby(att_X + att_Y + att_C)['Prob'].transform('sum')
    df_hts_XYC = df_hts_XYC.drop_duplicates()
    df_hts_XYC.rename(columns = {'Prob':'Prob_XYC_hts'}, inplace = True)

    # Subproblem P_{op_fusion}
    cluster_loss, df_hts_XYC_opt =  prob_XYC_optim(df_hts_XYC, df_cel_YC)
    print('Finish df_hts_XYC_opt')
    df_hts_XYC_opt = reduce_mem_usage(df_hts_XYC_opt)

    df_merge_XYZ = pd.merge(df_hts_XYC_opt, df_cel_YZC, on=att_Y + att_C)
    df_merge_XYZ = pd.merge(df_merge_XYZ, df_cel_YC, on=att_Y + att_C)
    df_merge_XYZ['Prob_XYZ'] = df_merge_XYZ['Prob_XYC_opt'] * df_merge_XYZ['Prob_YZC_cel'] / df_merge_XYZ['Prob_YC_cel']
    df_merge_XYZ = df_merge_XYZ.drop(columns=['Prob_XYC_opt', 'Prob_YZC_cel', 'Prob_YC_cel'])
    df_merge_XYZ[['Prob_XYZ']] = df_merge_XYZ[['Prob_XYZ']]/df_merge_XYZ[['Prob_XYZ']].sum()
    print('Prob_XYZ ', df_merge_XYZ[['Prob_XYZ']].sum())

    df_merge_XYZ = reduce_mem_usage(df_merge_XYZ)
    df_merge_XYZ.to_csv('output/P_XYZ_sim.csv', index = False)
    print(df_merge_XYZ.info())
    del df_merge_XYZ
    print('Clustering loss', cluster_loss)

#####################################################################

birch_threshold = 0.005
branching_factor = 10

# Initial clustering parameters
k_cluster = 32
n_partition = 2
w_cluster = 1.0

df_celK = pd.read_csv(project_path + 'data/data_cel_sim.csv')
df_htsK = pd.read_csv(project_path + 'data/data_hts_sim.csv')

df_htsK = df_htsK.drop(columns=['Distance'])
df_celK = df_celK.drop(columns=['Distance'])
add_distance(df_htsK, n_partition)
add_distance(df_celK, n_partition)
df_htsK = reduce_mem_usage(df_htsK)
df_celK = reduce_mem_usage(df_celK)

df_celK = df_celK[att_Y + att_HC + att_Z + ['Prob']]
df_celK['Prob'] = df_celK.groupby(att_Y + att_HC + att_Z)['Prob'].transform('sum')
df_celK = df_celK.drop_duplicates()

df_htsK_ = df_htsK[att_Z].drop_duplicates()
df_celK_ = df_celK[att_Z].drop_duplicates()
df_spatial_clustering = pd.concat([df_celK_, df_htsK_]).drop_duplicates()

# Subproblem P_{clustering}
clusters = clustering(df_spatial_clustering, birch_threshold,branching_factor, k_cluster, w_cluster)
print('Complete clustering...')

# Subproblem P_{op_fusion}
prob_XYZ(df_htsK, df_celK, clusters)

# Simulation

In [ ]:
import pandas as pd
import numpy as np
import time
import math
import warnings
warnings.filterwarnings('ignore')

# Attributes
att_X = ['Gender', 'Age', 'Homeincome', 'NumHH']
att_Z = ['Distance','Home_X', 'Home_Y' , 'Work_X', 'Work_Y']
att_C = ['Cluster']
att_HC = ['Home_Hcode', 'Work_Hcode']

# Get index from the true distribution for validation only
df_cel = pd.read_csv(project_path + 'data/data_true_sim.csv')
df_cel = df_cel[att_HC].reset_index()
df_cel['Indx'] = df_cel.index
df_cel = df_cel.drop(columns=['index'])
print(len(df_cel))

# Get the fused distribution
df_fus = pd.read_csv(project_path + 'output/P_XYZ_sim.csv')
df_fus = df_fus[att_X + att_Z + att_HC + ['Prob_XYZ']]
df_fus = reduce_mem_usage(df_fus)

# Chunk size for simulation
size = 1000
list_df_cel = [df_cel[i:i+size] for i in range(0,df_cel.shape[0],size)]
Z = math.ceil(len(df_cel)/size)
print('no. of chunks', Z)
df_final = pd.DataFrame()
for z in range(Z):
    df_fus_ = pd.merge(df_fus, list_df_cel[z], on= att_HC)
    if (len(df_fus_) > 0):
        df_fus_[['Prob_XYZ_']] = df_fus_[['Prob_XYZ']] / df_fus_[['Prob_XYZ']].sum()
        df_fus_ = df_fus_.reset_index()
        df_fus_['Indx2'] = df_fus_.index
        NN = int(4*len(df_fus_))
        pop = list(range(0, len(df_fus_)))
        prob = df_fus_['Prob_XYZ_'].to_numpy()
        samples = np.random.choice(pop, size=NN, replace=True, p=prob)
        df = pd.DataFrame({'Indx2': samples})
        df_fus_ = pd.merge(df, df_fus_, how='left', on=['Indx2'])
        df_fus_ = df_fus_.drop_duplicates(subset= ['Indx'], keep='first')
        df_fus_ = df_fus_[['Indx'] + att_X + att_HC + att_Z + ['Prob_XYZ']]
        df_fus_ = reduce_mem_usage(df_fus_)
        df_final = pd.concat([df_final, df_fus_])
        del df_fus_

df_final = reduce_mem_usage(df_final)
print(df_final.info())
# Save the simulation result to file
df_final.to_csv(project_path + 'output/sample_sim.csv', index=False)

# Fesibility, heterogenity, divergence

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import rel_entr, kl_div
import warnings
warnings.filterwarnings('ignore')

def js_div(p, q):
    h = 0.5*np.add(p,q)
    return 0.5*rel_entr(p, h) + 0.5*rel_entr(q, h)

df_true_ = pd.read_csv(project_path + 'data/data_true_sim.csv')
df_fus_ = pd.read_csv(project_path + 'output/sample_sim.csv')

# Attribute combinations
X = [['Gender', 'Age', 'Homeincome'], ['Gender', 'Age', 'NumHH'],  ['Gender', 'Homeincome', 'NumHH'], ['Age', 'Homeincome', 'NumHH']]
Z = [['Home_Hcode'], ['Work_Hcode']]
att = X[1] + Z[1]

df_fus = df_fus_[att]
df_fus['Numbers'] = 1.0
df_fus['Numbers'] = df_fus.groupby(att)['Numbers'].transform('sum')
df_fus = df_fus.drop_duplicates()
df_fus[['Prob_fus']] = df_fus[['Numbers']] / df_fus[['Numbers']].sum()
df_fus = df_fus.drop(columns=['Numbers'])

df_true = df_true_[att]
df_true['Numbers'] = 1.0
df_true['Numbers'] = df_true.groupby(att)['Numbers'].transform('sum')
df_true = df_true.drop_duplicates()
df_true[['Prob_true']] = df_true[['Numbers']] / df_true[['Numbers']].sum()
df_true = df_true.drop(columns=['Numbers'])

df0 = pd.merge(df_true, df_fus, how='outer', on=att)
df0 = df0.replace(np.nan, 0)
divg = np.sum(js_div(df0['Prob_true'], df0['Prob_fus']))

df_true__ = df_true[att]
df_fus__ = df_fus[att]

df1 = pd.merge(df_fus, df_true__.drop_duplicates(), on=att)
precision = df1[['Prob_fus']].sum()[0]
df2 = pd.merge(df_true, df_fus__.drop_duplicates(), on=att)
recal = df2[['Prob_true']].sum()[0]

print('feasibility (precision):', precision)
print('diversity (recal):', recal)
print('divergence:', divg)


feasibility (precision): 0.8208552138034508
diversity (recal): 0.7836000000000001
divergence: 0.1751403631231393


# Ploting sociodemographics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def statistics_gen(df_, att, s):
    NN = len(df_)
    df = df_[att]
    df['Numbers'] = 1.0
    df['Numbers'] = df.groupby(att)['Numbers'].transform('sum')
    df = df.drop_duplicates().reset_index(drop=True)
    df[['Pop_' + s]] = df[['Numbers']] / NN
    df = df.drop(columns=['Numbers'])
    return df

def statistics_gen2(df_, att, s):
    df = df_[att + ['Prob_XYZ']]
    df['Prob_XYZ'] = df.groupby(att)['Prob_XYZ'].transform('sum')
    df = df.drop_duplicates().reset_index(drop=True)
    df[['Pop_' + s]] = df[['Prob_XYZ']]
    df = df.drop(columns=['Prob_XYZ'])
    return df

def statistics(df_, att, s):
    df = df_[att + ['Prob']]
    df['Prob'] = df.groupby(att)['Prob'].transform('sum')
    df = df.drop_duplicates().reset_index(drop=True)
    df[['Pop_' + s]] = df[['Prob']]
    df = df.drop(columns=['Prob'])
    return df

df_gen = pd.read_csv(project_path + 'output/sample_sim.csv')
df_hts = pd.read_csv(project_path + 'data/data_hts_sim.csv')
df_true = pd.read_csv(project_path + 'data/data_true_sim.csv')

df_gen_Age = statistics_gen(df_gen,['Age'],'gen')
df_gen_Gender = statistics_gen(df_gen,['Gender'],'gen')
df_gen_Homeincome = statistics_gen(df_gen,['Homeincome'],'gen')
df_gen_NumHH = statistics_gen(df_gen,['NumHH'],'gen')

df_hts_Age = statistics(df_hts,['Age'],'hts')
df_hts_Gender = statistics(df_hts,['Gender'],'hts')
df_hts_Homeincome = statistics(df_hts,['Homeincome'],'hts')
df_hts_NumHH = statistics(df_hts,['NumHH'],'hts')

df_true_Age = statistics_gen(df_true,['Age'],'true')
df_true_Gender = statistics_gen(df_true,['Gender'],'true')
df_true_Homeincome = statistics_gen(df_true,['Homeincome'],'true')
df_true_NumHH = statistics_gen(df_true,['NumHH'],'true')

df_Age = pd.merge(df_gen_Age, df_hts_Age, on=['Age'])
df_Age = pd.merge(df_Age, df_true_Age, on=['Age'])

df_Gender = pd.merge(df_gen_Gender, df_hts_Gender, on=['Gender'])
df_Gender = pd.merge(df_Gender, df_true_Gender, on=['Gender'])

df_Homeincome = pd.merge(df_gen_Homeincome, df_hts_Homeincome, on=['Homeincome'])
df_Homeincome = pd.merge(df_Homeincome, df_true_Homeincome, on=['Homeincome'])

df_NumHH = pd.merge(df_gen_NumHH, df_hts_NumHH, on=['NumHH'])
df_NumHH = pd.merge(df_NumHH, df_true_NumHH, on=['NumHH'])

fig_x = 3
fig_y = 1.7
col_map = {'Pop_gen':'Generated', 'Pop_hts':'HTS', 'Pop_true':'Ground truth'}
lengends = ['HTS','Generated', 'Ground truth']
colors = ['darkcyan','tab:red', 'navajowhite', 'springgreen']

df_Homeincome.rename(columns = col_map, inplace = True)
df_Homeincome = df_Homeincome.sort_values(by=['Homeincome']).reset_index(drop=True)
df_Homeincome.index = [1,2,3,4,5,6]
df_Homeincome[lengends].plot.bar(rot=0,
            color=colors,
            figsize=(fig_x,fig_y),
            xlabel='Household income',
            legend=False,
            ylabel='Proportion'
            )

df_NumHH.rename(columns = col_map, inplace = True)
df_NumHH = df_NumHH.sort_values(by=['NumHH']).reset_index(drop=True)
df_NumHH.index = [1,2,3,4,5,6]
df_NumHH[lengends].plot.bar(rot=0,
            color=colors,
            figsize=(fig_x,fig_y),
            xlabel='Household size', legend=False,
            ylabel='Proportion'
            )

df_Age.rename(columns = col_map, inplace = True)
df_Age = df_Age.sort_values(by=['Age']).reset_index(drop=True)
df_Age.index = [1,2,3,4,5]
df_Age[lengends].plot.bar(rot=0,
            color=colors,
            figsize=(fig_x,fig_y),
            xlabel='Age', legend=False,
            ylabel='Proportion'
            )

df_Gender.rename(columns = col_map, inplace = True)
df_Gender = df_Gender.sort_values(by=['Gender']).reset_index(drop=True)
df_Gender.index = [1,2]
df_Gender[lengends].plot.bar(rot=0,
            color=colors,
            figsize=(fig_x,fig_y),
            xlabel='Gender', legend=True,
            ylabel='Proportion'
            )
plt.legend(loc='lower right')


# Ploting home-work locations

In [ ]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df_fus = pd.read_csv(project_path + 'output/sample_sim.csv')
df_true = pd.read_csv(project_path + 'data/data_true_sim.csv')
df_hts = pd.read_csv(project_path + 'data/data_hts_sim.csv')

def home_work(df_fus, df_hts, df_true, hw):
    N_fus = len(df_fus)
    home_fus = df_fus[[hw+'_Hcode']]
    home_fus['Numbers'] = 1.0
    home_fus['Numbers'] = home_fus.groupby([hw+'_Hcode'])['Numbers'].transform('sum')
    home_fus = home_fus.drop_duplicates()
    home_fus[['Prob_fus']] = home_fus[['Numbers']] /N_fus
    home_fus = home_fus.drop(columns=['Numbers'])

    home_hts = df_hts[[hw+'_Hcode', 'Prob']]
    home_hts['Prob'] = home_hts.groupby(hw+'_Hcode')['Prob'].transform('sum')
    home_hts = home_hts.drop_duplicates().reset_index(drop=True)
    home_hts[['Prob_hts']] = home_hts[['Prob']]
    home_hts = home_hts.drop(columns=['Prob'])

    N_true = len(df_true)
    home_true = df_true[[hw+'_Hcode']]
    home_true['Numbers'] = 1.0
    home_true['Numbers'] = home_true.groupby([hw+'_Hcode'])['Numbers'].transform('sum')
    home_true = home_true.drop_duplicates()
    home_true[['Prob_true']] = home_true[['Numbers']] /N_fus
    home_true = home_true.drop(columns=['Numbers'])

    ss = 'Population'
    if hw == 'Home':
        ss = 'Population'
    elif hw =='Work':
        ss = 'Number_of_employees'

    home = pd.merge(home_fus, home_hts,  how='outer', on=[hw+'_Hcode'])
    home = pd.merge(home, home_true,  how='outer', on=[hw+'_Hcode'])
    home = home.replace(np.nan, 0)
    home = home[['Prob_true', 'Prob_fus', 'Prob_hts']]
    home = home.sort_values(by=['Prob_true'], ascending=True).reset_index()
    home['Indx'] = home.index
    home = home.drop(columns=['index'])
    return home

def work_home_plot(df, hw):
    fig_x = 3
    fig_y = 1.7
    col_map = {'Pop_gen':'Generated', 'Pop_true':'Ground truth', 'Pop_hts':'HTS'}
    lengends = ['Generated', 'HTS', 'Ground truth']
    colors = ['tab:red', 'darkcyan', 'navajowhite', 'springgreen']
    markers = ['o', 'x', '^']

    ax = plt.gca()
    df.plot(kind='scatter', x='Indx', y='Prob_hts', marker = markers[1], alpha=1, ax = ax, s=7, c=colors[1], linewidth=0.5)
    df.plot(kind='scatter', x='Indx', y='Prob_fus', figsize=(fig_x,fig_y), alpha=1, marker=markers[0], ax = ax, s=5, c=colors[0], linewidth=0.5)
    df.plot(kind='scatter', x='Indx', y='Prob_true',marker = markers[2], alpha=1, ax = ax, s=3, c=colors[2], linewidth=0.5)

    ss = ''
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.set_xlabel('Location index ' + ss)
    ss = 'Prop. of residents'
    if hw =='Work':
        ss = 'Prop. of employees'
    ax.set_ylabel(ss)

home__ = home_work(df_fus, df_hts, df_true,'Home')
work__ = home_work(df_fus, df_hts, df_true,'Work')

work_home_plot(home__, 'Home')
#work_home_plot(work__, 'Work')